# Import Required Libraries

In [10]:
import emoji
import os
import glob
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [12]:
print(emoji.emojize('Phd is very easy!!! :thumbs_up:'))

AttributeError: module 'emoji' has no attribute 'emojize'

In [13]:
dir(emoji)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

# Change Working Directory and Concatenate Multiple Datasets into One

In [2]:
path = 'C:\\Users\\melin\\Documents\\Springboard Data Science Career Track\\Capstone Projects\\Capstone Project 2\\COVID-19 Tweets\\CP2data'

# Check current working directory.
retval = os.getcwd()
print("Current working directory %s" % retval)

# Now change the directory
os.chdir(path)

# Check current working directory.
retval = os.getcwd()

print("Directory changed successfully %s" % retval)

Current working directory C:\Users\melin
Directory changed successfully C:\Users\melin\Documents\Springboard Data Science Career Track\Capstone Projects\Capstone Project 2\COVID-19 Tweets\CP2data


In [3]:
#Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [4]:
#combine all files in the list
df_tweets = pd.concat([pd.read_csv(f) for f in all_filenames ])

# Explore Dataset

In [5]:
#Check first 5 rows of df_tweets dataset
df_tweets.head()

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang
0,1244051646071611394,860252856829587457,2020-03-29T00:00:00Z,IMSS_SanLuis,"Ante cualquier enfermedad respiratoria, no te ...",TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,1008,41,NaN,2017-05-04T22:00:38Z,False,es
1,1244051645039706112,1125933654943895553,2020-03-29T00:00:00Z,intrac_ccs,#ATENCIÓN En el Terminal Nuevo Circo se implem...,TweetDeck,NaN,NaN,NaN,False,...,1,NaN,NaN,NaN,90,316,NaN,2019-05-08T01:21:16Z,False,es
2,1244051645975191557,80943559,2020-03-29T00:00:00Z,rlieving,“People are just storing up. They are staying ...,TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,136,457,NaN,2009-10-08T21:06:08Z,False,en
3,1244051646750928897,817072420947247104,2020-03-29T00:00:00Z,Tu_IMSS_Coah,"Si empezaste a trabajar, necesitas dar de alta...",TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,1549,170,NaN,2017-01-05T18:17:00Z,False,es
4,1244051647032102914,788863557349670913,2020-03-29T00:00:00Z,Tabasco_IMSS,Una sociedad informada está mejor preparada an...,TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,868,125,NaN,2016-10-19T22:05:03Z,False,es


In [6]:
#Get summary of df_tweets dataframe
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14607013 entries, 0 to 355386
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   status_id             int64  
 1   user_id               int64  
 2   created_at            object 
 3   screen_name           object 
 4   text                  object 
 5   source                object 
 6   reply_to_status_id    float64
 7   reply_to_user_id      float64
 8   reply_to_screen_name  object 
 9   is_quote              bool   
 10  is_retweet            bool   
 11  favourites_count      int64  
 12  retweet_count         int64  
 13  country_code          object 
 14  place_full_name       object 
 15  place_type            object 
 16  followers_count       int64  
 17  friends_count         int64  
 18  account_lang          float64
 19  account_created_at    object 
 20  verified              bool   
 21  lang                  object 
dtypes: bool(3), float64(3), int64(6), object(1

In [7]:
#Check sum of missing values in each column of df_tweets dataframe
df_tweets.isnull().sum()

status_id                      0
user_id                        0
created_at                     0
screen_name                    2
text                           0
source                        83
reply_to_status_id      12861933
reply_to_user_id        12495830
reply_to_screen_name    12495830
is_quote                       0
is_retweet                     0
favourites_count               0
retweet_count                  0
country_code            13950294
place_full_name         13947685
place_type              13947685
followers_count                0
friends_count                  0
account_lang            14607013
account_created_at             0
verified                       0
lang                           0
dtype: int64

# Manually Remove Redundant Columns and Rows

In [8]:
#Remove rows in which text are not English
df_tweets = df_tweets[df_tweets.lang == 'en']

In [9]:
#Remove columns with missing percentage of 60% or more
df_tweets = df_tweets.loc[:, df_tweets.isnull().mean() < .6]

In [10]:
#Remove redundant columns 
df_tweets = df_tweets.drop(['status_id', 'user_id'], axis = 1)
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8133785 entries, 2 to 355386
Data columns (total 13 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   created_at          object
 1   screen_name         object
 2   text                object
 3   source              object
 4   is_quote            bool  
 5   is_retweet          bool  
 6   favourites_count    int64 
 7   retweet_count       int64 
 8   followers_count     int64 
 9   friends_count       int64 
 10  account_created_at  object
 11  verified            bool  
 12  lang                object
dtypes: bool(3), int64(4), object(6)
memory usage: 705.9+ MB


In [11]:
#Randomly drop additional 90% of rows  
df_tweets = df_tweets.sample(frac=.1)
df_tweets.shape

(813378, 13)

In [12]:
# Convert created_at and account_created_at variables from object to datetime64 
df_tweets['created_at'] = df_tweets['created_at'].astype('datetime64')
df_tweets['account_created_at'] = df_tweets['account_created_at'].astype('datetime64')
# Check for earliest and latest tweet
print("Earliest tweet: ", df_tweets['created_at'].min())
print("Latest tweet: ", df_tweets['created_at'].max(), '\n')

Earliest tweet:  2020-03-29 00:00:00
Latest tweet:  2020-04-30 23:59:57 



# Clean Twitter Text

In [13]:
# function for twitter text cleaning 
def tweet_cleaner(text):
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                  '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', text) #remove url links
    text = re.sub("(@[A-Za-z0-9_]+)","", text) #remove twitter handles (@user)
    # convert emojis to text
    text = emojis.decode(text)
    text = text.replace(":"," ")
    text = ' '.join(text.split())
    # remove punctuations, numbers, and special characters 
    text = re.sub("[^a-zA-Z'_]"," ",text) #include contractions with "'" and emoji texts with "_"
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase
    text = text.lower()
    return text

In [14]:
#Apply text cleaning function to cleaned_text column in df_tweets dataframe
df_tweets['cleaned_text'] = df_tweets['text'].apply(lambda x: tweet_cleaner(x))

AttributeError: module 'emoji' has no attribute 'demojize'

In [ ]:
# function for lemmatizing words
def lemmatize_text(text):
    wl = WordNetLemmatizer()
    token_words=word_tokenize(str(text))
    token_words
    lemmatize_text=[]
    for word in token_words:
        lemmatize_text.append(wl.lemmatize(word))
        lemmatize_text.append(" ")
    return "".join(lemmatize_text)

In [ ]:
#Apply lemmatizing text function to cleaned_text column in df_tweets dataframe
df_tweets['cleaned_text'] = df_tweets['cleaned_text'].apply(lambda x: lemmatize_text(x))

In [ ]:
#Remove stop words in cleaned_text column

# function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    no_stopword_text = [w for w in str(text).split() if not w in stop_words]
    return ' '.join(no_stopword_text)

In [ ]:
#Apply the remove stop words function to cleaned_text column in df_tweets dataframe
df_tweets['cleaned_text'] = df_tweets['cleaned_text'].apply(lambda x: remove_stopwords(x))

In [ ]:
df_tweets.head()

# Export Final Dataset into csv

In [ ]:
df_tweets.to_csv (r'C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 2/df_tweets.csv', index = False, header=True)